# Draw Interst Area Boundaries On Videos: ffmpeg overlay

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import cv2
import os
import glob
import numpy as np
import time
import pdb

# Generate Polygon with Transparent Background

In [2]:
original_h = 1080
original_w = 1920

cameras = [1, 2]
for camera_index in cameras:
    # Define the polygon of Whole Interest Area for videos from Camera1 or Camera2
    if camera_index == 1:
        # crop frame: Camera1
        top_edge = int(original_h*(1/10))
        down_edge = int(original_h*1)
        left_edge = int(original_w*(1/5))
        right_edge = int(original_w*(4/5))
        point_1 = [left_edge, top_edge]
        point_2 = [left_edge, down_edge]
        point_3 = [right_edge, down_edge]
        point_4 = [right_edge, top_edge]
        whole_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])
    elif camera_index == 2:
        # crop frame: Camera2
        top_edge = int(original_h*(1/10))
        down_edge = int(original_h*(4/5))
        left_edge = int(original_w*(2.5/5))
        right_edge = int(original_w*(1))
        point_1 = [left_edge, top_edge]
        point_2 = [left_edge, down_edge]
        point_3 = [right_edge, down_edge]
        point_4 = [right_edge, top_edge]
        whole_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])
    else:
        # crop frame: test video
        top_edge = int(original_h*(1/10))
        down_edge = int(original_h*1)
        left_edge = int(original_w*(1/5))
        right_edge = int(original_w*(4/5))
        print('Polygon: Video not from Camera1 or Camera2!')
        point_1 = [left_edge, top_edge]
        point_2 = [left_edge, down_edge]
        point_3 = [right_edge, down_edge]
        point_4 = [right_edge, top_edge]
        whole_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])

    # Define the polygon of Core Interest Area for videos from Camera1 or Camera2
    cropped_w = right_edge - left_edge
    cropped_h = down_edge - top_edge
    if camera_index == 1:
        # polygon for Camera1
        point_1 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.20 * cropped_h)]
        point_2 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.62 * cropped_h)]
        point_3 = [left_edge + int(0.44 * cropped_w), top_edge + int(0.82 * cropped_h)]
        point_4 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.72 * cropped_h)]
        point_5 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.20 * cropped_h)]
        core_interest_area_polygon = np.array([point_1,point_2,point_3,point_4,point_5])
    elif camera_index == 2:
        # polygon for Camera2
        point_1 = [left_edge + int(0.15 * cropped_w), top_edge + int(0.05 * cropped_h)]
        point_2 = [left_edge + int(0.15 * cropped_w), top_edge + int(0.65 * cropped_h)]
        point_3 = [left_edge + int(0.95 * cropped_w), top_edge + int(0.75 * cropped_h)]
        point_4 = [left_edge + int(0.95 * cropped_w), top_edge + int(0.05 * cropped_h)]
        core_interest_area_polygon = np.array([point_1,point_2,point_3,point_4])
    else:
        # polygon for test video
        point_1 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.20 * cropped_h)]
        point_2 = [left_edge + int(0.17 * cropped_w), top_edge + int(0.62 * cropped_h)]
        point_3 = [left_edge + int(0.44 * cropped_w), top_edge + int(0.82 * cropped_h)]
        point_4 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.72 * cropped_h)]
        point_5 = [left_edge + int(0.61 * cropped_w), top_edge + int(0.20 * cropped_h)]
        print('Polygon: Video not from Camera1 or Camera2!')
        core_interest_area_polygon = np.array([point_1,point_2,point_3,point_4,point_5])

    # Define a transparent background
    n_channels = 4
    transparent_img = np.zeros((original_h, original_w, n_channels))
    
    cv2.drawContours(transparent_img, [whole_interest_area_polygon], -1, (255, 0, 0, 255), 6, cv2.LINE_AA) # BGR
    cv2.drawContours(transparent_img, [core_interest_area_polygon], -1, (0, 0, 255, 255), 6, cv2.LINE_AA)

    if camera_index == 1:
        cv2.putText(transparent_img, "Camera View", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255, 255), 4)
        cv2.putText(transparent_img, "Whole Interest Area", (left_edge+10, top_edge+80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0, 255), 4)
        cv2.putText(transparent_img, "Core Interest", (point_1[0]+10, point_1[1]+80), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255, 255), 4)
        cv2.putText(transparent_img, "Area", (point_1[0]+10, point_1[1]+130), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255, 255), 4)
    elif camera_index == 2:
        cv2.putText(transparent_img, "Camera View", (800, 1000), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255, 255), 4)
        cv2.putText(transparent_img, "Whole Interest Area", (left_edge+10, down_edge-50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 0, 0, 255), 4)
        cv2.putText(transparent_img, "Core Interest Area", (point_2[0]+10, point_2[1]-50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255, 255), 4)
    else:
        pass
    
    cv2.imwrite('camera{}_transparent_img.png'.format(camera_index), transparent_img)
    print('Generated: {}'.format('camera{}_transparent_img.png'.format(camera_index)))

Generated: camera1_transparent_img.png
Generated: camera2_transparent_img.png


# Using "ffmpeg overlay" Processing Videos 

In [3]:
raw_video_dir = '../ROM_raw_videos'
raw_video_sub_dir = os.listdir(raw_video_dir)
raw_video_sub_dir.sort()
raw_video_sub_dir

['Oct_02',
 'Oct_03',
 'Oct_05',
 'Sep_12',
 'Sep_13',
 'Sep_14',
 'Sep_17',
 'Sep_18',
 'Sep_19',
 'Sep_20',
 'Sep_21',
 'Sep_24',
 'Sep_25',
 'Sep_26',
 'Sep_27',
 'Sep_28']

In [4]:
output_video_dir = '../ROM_raw_videos_with_interst_area_ffmpeg'

In [5]:
raw_video_sub_dir = ['Sep_17']
for sub_dir in raw_video_sub_dir:
    files_path = glob.glob(os.path.join(raw_video_dir, sub_dir, '*.mp4'))
    print('{} files in {}'.format(len(files_path), sub_dir))
    
    for f_path in files_path:
        if 'Camera1' in f_path and '1300' in f_path:
            # Load overlay image
            if 'Camera1' in f_path:
                overlay_image = 'camera1_transparent_img.png'
            else:
                overlay_image = 'camera2_transparent_img.png'

            # Define input filename
            input_filename = f_path

            # Define output filename
            output_video_sub_dir = os.path.join(output_video_dir, sub_dir)
            if not os.path.exists(output_video_sub_dir):
                os.makedirs(output_video_sub_dir)
            f_name = f_path.split('/')[-1]
            output_video_filename = os.path.join(output_video_sub_dir,'{}_with_interest_area.avi'.format(f_name.split('.mp4')[0]))
            print('Processing: {}'.format(output_video_filename.split('/')[-1]))
            start_time = time.time()
            # ffmpeg overlay
            code = os.system("ffmpeg -i {} -i {} -filter_complex \"[0:v][1:v] overlay=0:0\" -c:a copy {}".format(input_filename,overlay_image,output_video_filename))
            print('code: {}, Elapsed time: {}s'.format(code, time.time() - start_time))
                                                                                                                              

6 files in Sep_17
Processing: Camera1_Sep_17_1300_1400_Parameterized_Learning_Agent_with_interest_area.avi
code: 0, Elapsed time: 1827.0604043006897s
